In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import glob
import os
import rdkit
import concurrent.futures
import random
from scipy import stats
from sklearn import preprocessing
from skopt import BayesSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from sklearn.metrics import matthews_corrcoef, precision_recall_curve, accuracy_score, auc
from sklearn.model_selection import cross_val_predict
from sklearn.neural_network import MLPClassifier
from sklearn.utils import parallel_backend
from xgboost.sklearn import XGBClassifier
from rdkit import Chem
from rdkit.Chem import AllChem
import deepchem as dc
from deepchem.utils import download_url, load_from_disk
from deepchem.utils.vina_utils import prepare_inputs
from joblib import Parallel, delayed
import tempfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, regularizers
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Activation
from tensorflow.keras.optimizers import Adadelta, Adam, RMSprop
import hyperopt
from hyperopt import hp, tpe, Trials, fmin, STATUS_OK, space_eval
from pandas.core.frame import DataFrame

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import Data

# Load the features

In [ ]:
train_plec=np.loadtxt('/home/pathway/train_plec.txt')
test_plec=np.loadtxt('/home/pathway/test_plec.txt')

In [ ]:
train_df=pd.read_csv('/home/pathway/train.csv') 
test_df=pd.read_csv('/home/pathway/test.csv')

In [ ]:
Train_class=train_df['activity']
Test_class=test_df['activity']
# The labels for PLEC-ML

In [ ]:
train_conv = np.load('/home/wangbo/cgas_new/Feature/Split_data2/train_ConvMol.npz', allow_pickle=True)
test_conv = np.load('/home/wangbo/cgas_new/Feature/Split_data2/test_ConvMol.npz', allow_pickle=True)
train_adj = train_conv['adjacency_matrices']
test_adj = test_conv['adjacency_matrices']
train_node = train_conv['node_features']
test_node = test_conv['node_features']

In [ ]:
label_mapping = {'Active': 1, 'Inactive': 0}
train_df['activity'] = train_df['activity'].map(label_mapping)
test_df['activity'] = test_df['activity'].map(label_mapping)
train_labels = torch.tensor(train_df['activity'].values, dtype=torch.long)
test_labels = torch.tensor(test_df['activity'].values, dtype=torch.long)
# The labels for ConvMol-GCN

# PLEC-ML

### RF algorithm

In [ ]:
PR_AUC_rf=[]
for i in range(10):
    rf_model=RandomForestClassifier(n_estimators=400,max_features='sqrt',n_jobs=30)
    rf_model.fit(train_plec,Train_class)

    predicition_test_rf_class=rf_model.predict(test_plec)
    predicition_test_rf_prob=rf_model.predict_proba(test_plec)

    feature_result_rf=pd.DataFrame({'Active_Prob':predicition_test_rf_prob[:,0],
                                'Inactive_Prob':predicition_test_rf_prob[:,1],
                                'Predicted_Class':predicition_test_rf_class,
                                'Real_Class':Test_class})

    precision_rf,recall_rf,threshold_rf=precision_recall_curve(Test_class,feature_result_rf['Active_Prob'],pos_label='Active')
    pr_auc_rf=auc(recall_rf,precision_rf)

    PR_AUC_rf.append(pr_auc_rf)

    print(f'{i+1}-PR_AUC:{pr_auc_rf}')
PR_AUC=pd.DataFrame(PR_AUC_rf,columns=['RF'])

In [ ]:
rf_precision_recall=pd.DataFrame({'Precision':precision_rf,'Recall':recall_rf})
feature_result_rf.to_csv('/home/pathway/rf_result.csv',index=False)
rf_precision_recall.to_csv('/home/pathway/rf_precison_recall.csv',index=False)

### XGB

In [ ]:
PR_AUC_xgb=[]
for i in range(10):
    label_mapping={'Active':1,'Inactive':0}
    Train_class_encoded=np.array([label_mapping[label] for label in Train_class])
    
    xgb_model=XGBClassifier(n_jobs=40,subsample=np.random.uniform(low=0.5,high=1.0))
    xgb_model.fit(np.array(train_plec),Train_class_encoded)

    predicition_test_xgb_class=xgb_model.predict(np.array(test_plec))
    predicition_test_xgb_prob=xgb_model.predict_proba(np.array(test_plec))

    inverse_label_mapping={v:k for k,v in label_mapping.items()}
    predicted_labels=np.array([inverse_label_mapping[pred] for pred in predicition_test_xgb_class])

    feature_result_xgb=pd.DataFrame({'Active_Prob':predicition_test_xgb_prob[:,1],
                                'Inactive_Prob':predicition_test_xgb_prob[:,0],
                                'Predicted_Class':predicted_labels,#predicition_test_xgb_feature_class,
                                'Real_Class':Test_class})
    precision_xgb,recall_xgb,threshold_xgb=precision_recall_curve([label_mapping[label] for label in Test_class],feature_result_xgb['Active_Prob'],pos_label=1)
    xgb_precison_recall=pd.DataFrame({'Precision':precision_xgb,'Recall':recall_xgb})

    pr_auc_xgb=auc(recall_xgb,precision_xgb)

    PR_AUC_xgb.append(pr_auc_xgb)

    print(f'{i+1}-PR_AUC:{pr_auc_xgb}')

In [ ]:
xgb_precision_recall=pd.DataFrame({'Precision':precision_xgb,'Recall':recall_xgb})
feature_result_xgb.to_csv('/home/pathway/xgb_result.csv',index=False)
xgb_precision_recall.to_csv('/home/pathway/xgb_precison_recall.csv',index=False)

### ANN

In [ ]:
PR_AUC_ann=[]
for i in range(10):
    ann_model=MLPClassifier(max_iter=9000)
    ann_model.fit(train_plec,Train_class)

    predicition_test_ann_class=ann_model.predict(test_plec)
    predicition_test_ann_prob=ann_model.predict_proba(test_plec)

    feature_result_ann=pd.DataFrame({'Active_Prob':predicition_test_ann_prob[:,0],
                                'Inactive_Prob':predicition_test_ann_prob[:,1],
                                'Predicted_Class':predicition_test_ann_class,
                                'Real_Class':Test_class})

    precision_ann,recall_ann,threshold_ann=precision_recall_curve(Test_class,feature_result_ann['Active_Prob'],pos_label='Active')
    pr_auc_ann=auc(recall_ann,precision_ann)
    PR_AUC_ann.append(pr_auc_ann)

    print(f'{i+1}-PR_AUC:{pr_auc_ann}')

In [ ]:
ann_precision_recall=pd.DataFrame({'Precision':precision_ann,'Recall':recall_ann})
feature_result_ann.to_csv('/home/pathway/ann_result.csv',index=False)
ann_precision_recall.to_csv('/home/pathway/ann_precison_recall.csv',index=False)

### SVM

In [ ]:
PR_AUC_svm=[]
for i in range(10):
    random_seed=np.random.randint(0,1000)
    train_plec,Train_class=shuffle(train_plec,Train_class,random_state=random_seed)
    test_plec,Test_class=shuffle(test_plec,Test_class,random_state=random_seed)
    svm_model=SVC(degree=3,kernel='rbf',probability=True,random_state=random_seed)
    svm_model.fit(train_plec,Train_class)

    predicition_test_svm_class=svm_model.predict(test_plec)
    predicition_test_svm_prob=svm_model.predict_proba(test_plec)

    feature_result_svm=pd.DataFrame({'Active_Prob':predicition_test_svm_prob[:,0],
                                'Inactive_Prob':predicition_test_svm_prob[:,1],
                                'Predicted_Class':predicition_test_svm_class,
                                'Real_Class':Test_class})

    precision_svm,recall_svm,threshold_svm=precision_recall_curve(Test_class,feature_result_svm['Active_Prob'],pos_label='Active')
    pr_auc_svm=auc(recall_svm,precision_svm)
    PR_AUC_svm.append(pr_auc_svm)

    print(f'{i+1}-PR_AUC:{pr_auc_svm}')

In [ ]:
svm_precision_recall=pd.DataFrame({'Precision':precision_svm,'Recall':recall_svm})
feature_result_svm.to_csv('/home/wangbo/pathway/svm_result.csv',index=False)
svm_precision_recall.to_csv('/home/wangbo/pathway/svm_precison_recall.csv',index=False)

# ConvMol-GCN

### GCN

In [ ]:
def convert_to_torch_geometric_with_labels(adjacency_matrices, node_features, labels):
    torch_geometric_data_list = []
    for adj, features, label in zip(adjacency_matrices, node_features, labels):
        edge_index = torch.tensor(np.array(np.nonzero(adj)), dtype=torch.long)
        x = torch.tensor(features, dtype=torch.float)
        batch = torch.zeros(x.size(0), dtype=torch.long)  # 单图情况，batch全为0
        data = Data(x=x, edge_index=edge_index, y=label, batch=batch)
        torch_geometric_data_list.append(data)
    return torch_geometric_data_list

In [ ]:
train_data_list = convert_to_torch_geometric_with_labels(train_adj, train_node, train_labels)
test_data_list = convert_to_torch_geometric_with_labels(test_adj, test_node, test_labels)
train_data_list, val_data_list, train_labels_split, val_labels_split = train_test_split(
train_data_list, train_labels, test_size=0.2, random_state=42)

In [ ]:
class GCNModel(torch.nn.Module):
    def __init__(self, num_node_features):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(num_node_features, 64)
        self.conv2 = GCNConv(64, 64)
        self.fc1 = torch.nn.Linear(64, 128)
        self.fc2 = torch.nn.Linear(128, 1)  
        self.dropout = torch.nn.Dropout(0.5)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, data.batch)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)  
        return torch.sigmoid(x).squeeze()  

In [ ]:
def train(model, train_loader, optimizer, device, num_epochs=100, val_loader=None):
    model.train()
    best_val_loss = float('inf')
    best_model_state = None

    for epoch in range(num_epochs):
        epoch_loss = 0
        model.to(device)
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data)
            loss = F.binary_cross_entropy(out, data.y.float())  
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)
            optimizer.step()
            epoch_loss += loss.item()

        if val_loader:
            val_loss = 0
            all_val_y_true = []
            all_val_y_scores = []
            model.eval()
            with torch.no_grad():
                for val_data in val_loader:
                    val_data = val_data.to(device)
                    val_out = model(val_data)
                    val_loss += F.binary_cross_entropy(val_out, val_data.y.float()).item()
                    all_val_y_true.extend(val_data.y.cpu().numpy())
                    all_val_y_scores.extend(val_out.cpu().numpy())
            
            val_pr_auc = auc(
                precision_recall_curve(all_val_y_true, all_val_y_scores, pos_label=1)[1::-1]
            )  

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model_state = model.state_dict()

    return best_model_state,val_pr_auc

def test(model, data_loader, device):
    model.eval() 
    all_y_true = []
    all_y_scores = []

    with torch.no_grad():
        for data in data_loader:
            data = data.to(device)
            out = model(data) 

            y_true = data.y.cpu().numpy() 
            y_scores = out.cpu().numpy()  

            all_y_true.extend(y_true)
            all_y_scores.extend(y_scores)

    return np.array(all_y_true), np.array(all_y_scores)

In [ ]:
def run_experiment(train_data_list,test_data_list, num_runs=10,device):
    PR_AUC_gcn = []
    val_pr_auc_results = []
    num_node_features = train_data_list[0].x.shape[1]  
    num_classes = 2  

    for run in range(num_runs):
        torch.manual_seed(run)  

        train_loader = DataLoader(train_data_list, batch_size=32, shuffle=True)
        test_loader = DataLoader(test_data_list, batch_size=32, shuffle=False)

        model = GCNModel(num_node_features=num_node_features, num_classes=num_classes)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

        train(model, train_loader, optimizer, num_epochs=100)

        y_true, y_scores = test(model, test_loader)

        # 计算PR-AUC
        precision, recall, _ = precision_recall_curve(y_true, y_scores, pos_label=1)
        pr_auc = auc(recall, precision)
        PR_AUC_gcn.append(pr_auc)

        print(f'Run {run + 1} - PR-AUC: {pr_auc}')
        
    return PR_AUC_gcn,y_true, y_scores

In [ ]:
PR_AUC_gcn,y_true, y_scores = run_experiment(train_data_list, train_labels, num_runs=10,device)

In [ ]:
predicted_class = np.where(y_scores == 1, 1, 0)
feature_result_gcn = pd.DataFrame({
    'Active_Prob': y_scores,
    'Inactive_Prob': 1 - y_scores,
    'Predicted_Class': np.where(predicted_class == 1, 'Active', 'Inactive'),
    'Real_Class': np.where(y_true == 1, 'Active', 'Inactive')
})

In [ ]:
precision_gcn, recall_gcn, threshold_gcn = precision_recall_curve(y_true, y_scores, pos_label=1)
gcn_precision_recall=pd.DataFrame({'Precision':precision_gcn,'Recall':recall_gcn})
gcn_precision_recall.to_csv('/home/pathway/gcn_precison_recall.csv',index=False)
feature_result_gcn.to_csv('/home/pathway/gcn_result.csv',index=False)

# Save the results

In [ ]:
PR_AUC=DataFrame(pr_auc_rf)
PR_AUC.rename(columns={0:'RF'},inplace=True)
PR_AUC['ANN']=PR_AUC_ann
PR_AUC['XGB']=PR_AUC_xgb
PR_AUC['SVM']=PR_AUC_svm
PR_AUC['GCN']=PR_AUC_gcn

In [ ]:
PR_AUC.to_csv('/home/pathway/PR_AUC_10.csv',index=None)